# MyTrain

## DataSetInitialization

In [ ]:
import sys
import logging
from logging import getLogger
from recbole.utils import init_logger, init_seed
from Modules.mamba4poi import Mamba4POI
from recbole.config import Config
from utils import *
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
    set_color,
    get_flops,
    get_environment,
)
import torch
from Modules.myutils import * 

config = Config(model=Mamba4POI, config_file_list=['config.yaml'])
dataset = create_dataset(config)
train_data,valid_data,test_data = data_preparation(config, dataset)


In [ ]:
k=50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_user=(dataset.inter_feat[dataset.uid_field].max()+1).astype(int)
num_category=(dataset.item_feat["venue_category_id"].max()+1).astype(int)
num_POI=(dataset.item_feat["venue_id"].max()+1).astype(int)

Ms=torch.zeros((num_user,num_category),dtype=torch.float32)
_,M0,_,_=counting4all(train_data._dataset,device)

Ec=torch.zeros((num_category,k),dtype=torch.float32)
ESu=torch.zeros((num_user,k),dtype=torch.float32)

itemX=(dataset.item_feat["longitude"]).to_numpy()
itemY=(dataset.item_feat["latitude"]).to_numpy()
itemC=(dataset.item_feat["venue_category_id"]).to_numpy().astype(np.int)
Locations=torch.tensor(np.stack((itemX,itemY),axis=-1),dtype=torch.double)


row_nonzero_mask=M0.sum(1)>0
col_nonzero_mask=M0.sum(0)>0
nonzero_M0=M0[row_nonzero_mask][:,col_nonzero_mask]

E0u,S0,E0c=UC_SVD(nonzero_M0,k)

Ec[col_nonzero_mask]=E0c

EPOI=torch.einsum('ij,ik->ijk',Ec[itemC],Locations)

In [ ]:
for batch in train_data:
    userids,item_seqs,Xs,Ys,Cs,labels,times,negs=batch
    break

In [ ]:

Ms=accumulate_category(Cs,userids,Ms)
s_row_nonzero_mask=Ms.sum(1)>0
s_col_nonzero_mask=Ms.sum(0)>0
nonzero_Ms=Ms[s_row_nonzero_mask][:,s_col_nonzero_mask]

row,col=nonzero_Ms.shape
size = max(row, col)  
expanded_Ms = torch.zeros(size, size)
expanded_Ms[:row,:col]=nonzero_Ms
if row==size:
    expanded_Ms[:,col:]=torch.eye(row,row - col) * 1e5
else:
    expanded_Ms[row:,:]=torch.eye(col-row,col)*1e5

Esu,Ss,Esc=UC_SVD(expanded_Ms,k)

Estu=Ec[s_col_nonzero_mask]@Esc.T@Esu
ESu[s_row_nonzero_mask]=Estu[:row]

In [ ]:
location=np.stack((Xs,Ys),axis=-1)
El=torch.tensor(location,dtype=torch.double)
Ep=torch.einsum('usj,usk->usjk',Ec[Cs],El)#numuserXnumseqXcategoryfeatureXLocationfeature
S=torch.einsum('ut,usjk->ustjk',ESu[userids[:,0]],Ep)#numuserXnumseqXcategoryfeatureXLocationfeatureXUserFeat
labels=torch.tensor(np.array(labels))
valid_S=S[(labels==1)|(labels==2)].reshape(S.shape[0],-1,S.shape[2],S.shape[3],S.shape[4])


In [ ]:
Times=torch.tensor(times,dtype=torch.double)
dT=(Times[:,1:]-Times[:,:-1]).unsqueeze(2).unsqueeze(2).unsqueeze(2)
S1=valid_S[:,1:]/dT
S2=valid_S[:,:-1]/dT

shape = S1.shape

new_shape = (shape[0], shape[1] * 2, shape[2], shape[3], shape[4])

Ss = torch.zeros(new_shape,dtype=torch.double)
Ss[:,0::2]=S2
Ss[:,1::2]=S1


In [ ]:
import gc
torch.set_printoptions(profile="full")
for obj in gc.get_objects():
    if torch.is_tensor(obj) and obj.is_cuda:
        print(type(obj), obj.size())


# HyperGraphModel

#### notes:
We use a **HyperGraph Neural Network** to model the complex interactions among users, categories, and POIs, calculating the weight of hyperedges to reflect users' **interests in both categories and POIs**.<br>
Unlike our sequential prediction model, which predict user interest in the dynamic time sequence, the HGNN addresses the issue of dissimilar interests in the same POI caused by users' varied **breadth of interests** despite having equal interaction levels,by modeling static user profiles.The **distinction** between the two modules is determined by their approach to **utilizing interactions**: ***dynamically or statically***<br>
#### Nodes defination:
We detatch category from POI features to modeling its latent<br> relation between both POI and user<br><br>
**User**:       10dims for 3 features<br>
*num_poi_interaction*:int,*num_category_interaction*:int<br>
***active_area:tensor***(4,2) <--training target<br><br>
**Category**:   1dim for 1 feature<br>
*num_poi*:int,sigma:float(2,),center:float(2,)<br>
***using poi geographic distribution feature to reflect the feature of category***<br>
<br><br>
**POI**:        2dims for 1feature<br>
*location*:tensor(2,)<br>

             

#### Weight updating policy:
Our Hyperedge connected with three varied type nodes,whose weight can't be fully normalized as scalar,are designed with vector weight.Using attention mechanism to reflect the diffrent impact from nodes.



In [ ]:
from scipy.spatial import KDTree
def CreateAdjacencyList(locations):
    radius=200
    POI_tree=KDTree(locations)
    adjacency_list = POI_tree.query_ball_point(locations,r=radius,p=1)

    return adjacency_list

def adjacency_list_to_edge_index(adjacency_list,itemC,num_poi):
    #GroupIdx=[]
    edge_index = []
    catedge_index = []
    C=itemC+num_poi
    for src_node, neighbors in enumerate(adjacency_list):
       #GroupIdx.append(len(edge_index))
        for tgt_node in neighbors:
                edge_index.append([src_node, tgt_node])
        #GroupIdx.append(len(edge_index)-GroupIdx[-1])
        catedge_index.append([src_node,C[src_node]])
        catedge_index.append([C[src_node],src_node])
    turnidx=len(edge_index)
    edge_index = torch.tensor(edge_index+catedge_index, dtype=torch.int)
    return edge_index.T,turnidx#,GroupIdx

locations=np.stack((itemX,itemY),axis=-1)
num_poi=locations.shape[0]
adjacency_list=CreateAdjacencyList(locations)
CnitemC=np.concatenate([itemC, np.arange(num_category)])
edge_index,turnidx= adjacency_list_to_edge_index(adjacency_list,CnitemC,num_poi)

In [ ]:
print(edge_index.shape)

In [ ]:
import torch
import torch.nn as nn
edgeMLP = nn.Sequential(
            nn.Linear(10+50+2, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        ).to(device)
node_feats = torch.randn(location.shape[0]+num_POI, 10, requires_grad=True).to(device) # 可学习嵌入
locations = torch.tensor(location, dtype=torch.float32).to(device)
edge_index=edge_index.to(device)
Ec=Ec.to(device)
CnitemC=torch.tensor(CnitemC).to(device)
for idx in range(0,turnidx,1000):
            loc_diff=torch.abs(locations[edge_index[idx:min(idx+1000,turnidx),0]]- locations[edge_index[idx:min(idx+1000,turnidx),1]])
            cat_diff=torch.abs(Ec[CnitemC[edge_index[idx:min(idx+1000,turnidx),0]]]-Ec[CnitemC[edge_index[idx:min(idx+1000,turnidx),1]]])
            embedding_diff = torch.abs(node_feats[edge_index[idx:min(idx+1000,turnidx),0]] - node_feats[edge_index[idx:min(idx+1000,turnidx),1]])
            
            edge_feat=torch.cat([loc_diff,cat_diff,embedding_diff], dim=1)
            
       
            break

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing

def PositionEmbedding(positions, output_dim, noise_std=1, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    
    # 原始位置张量
    N, input_dim = positions.shape

    
    normalized_positions=positions/positions.max(0).values

    # 添加高斯噪声
    noise = torch.randn(N,output_dim-input_dim) * noise_std
    el = torch.concat([normalized_positions,noise],dim=1)
    
    return el
# -------------------------------
# 1. 超图卷积层
# -------------------------------
class HeteroHyperGCN(MessagePassing):
    def __init__(self, node_dim, cat_dim, hidden_dim, edge_dim, output_dim,CnitemC,batchsize):
        super(HeteroHyperGCN, self).__init__(aggr='add')  # 聚合方式为求和
        self.node_dim = node_dim
        self.cat_dim = cat_dim
        self.edge_dim = edge_dim
        self.hidden_dim = hidden_dim
        self.CnitemC=CnitemC
        self.BatchSize=batchsize


        # 边权重更新 MLP
        self.poiMLP = nn.Sequential(
            nn.Linear(2*node_dim+cat_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    

    
    def forward(self, node_feats, edge_index,turnidx,locations,Ec):
        edge_weights = self.compute_edge_weights(node_feats, edge_index,turnidx,locations,Ec)
        updated_feats = self.propagate(edge_index, x=node_feats, edge_weight=edge_weights)
        return updated_feats, edge_weights

    def compute_edge_weights(self, node_feats, edge_index,turnidx,locations,Ec):
        edge_weights = []
        for idx in range(0,turnidx,1000):
            end_idx=min(idx+1000,turnidx)
            loc_diff=torch.abs(locations[edge_index[0,idx:end_idx]]- locations[edge_index[1,idx:end_idx]])
            cat_diff=torch.abs(Ec[self.CnitemC[edge_index[0,idx:end_idx]]]-Ec[self.CnitemC[edge_index[1,idx:end_idx]]])
            embedding_diff =node_feats[edge_index[0,idx:end_idx]] - node_feats[edge_index[1,idx:end_idx]]
            
            edge_feat=torch.cat([loc_diff,cat_diff], dim=1)
            
            weight=embedding_diff*self.poiMLP(edge_feat)

            edge_weights.append(weight)

        for idx in range(turnidx,locations.shape[0],1000):
            end_idx=min(idx+1000,turnidx)
            embedding_diff = torch.abs(node_feats[edge_index[0,idx:end_idx]] - node_feats[edge_index[1,idx:end_idx]])
            decay=0.01
            weight=decay*embedding_diff
            edge_weights.append(weight)


        return torch.cat(edge_weights, dim=0)

    def message(self,x_j, edge_weight=None):
        return edge_weight 

    def aggregate(self, inputs, index):
        return torch.scatter(inputs, index, dim=0, reduce=self.aggr)

    def update(self, inputs,x):
        # 更新节点嵌入
        return inputs+x


# -------------------------------
# 2. 模型定义
# -------------------------------
class HeteroGraphModel(nn.Module):
    def __init__(self, node_dim, cat_dim, hidden_dim, edge_dim, output_dim,CnitemC,batchsize=1000):
        super(HeteroGraphModel, self).__init__()
        self.hyper_gcn = HeteroHyperGCN(node_dim, cat_dim, hidden_dim, edge_dim, output_dim,CnitemC,batchsize)
        self.embedding = nn.Embedding(node_dim, output_dim)

    def forward(self, node_feats, edge_index,turnidx,locations,Ec):
        node_feats, edge_weights = self.hyper_gcn(node_feats, edge_index,turnidx,locations,Ec)
        return node_feats, edge_weights

    def loss(self, node_feats, edge_index,turnidx,locations,Ec):
       total_loss=0
       for idx in range(0,turnidx,1000):
            end_idx=min(idx+1000,turnidx)
            loc_diff=torch.abs(locations[edge_index[idx:end_idx,0]]- locations[edge_index[idx:end_idx,1]])
            cat_diff=torch.abs(Ec[self.CnitemC[edge_index[idx:end_idx,0]]]-Ec[self.CnitemC[edge_index[idx:end_idx,1]]])
            embedding_diff = torch.abs(node_feats[edge_index[idx:end_idx,0]] - node_feats[edge_index[idx:end_idx,1]])
            batch_loss=embedding_diff/(loc_diff/cat_diff)
            total_loss += batch_loss
            


# -------------------------------
# 3. 数据加载与初始化
# -------------------------------
torch.cuda.empty_cache()

node_dim=10, cat_dim=50 ,hidden_dim=32, edge_dim=1, output_dim=20

locations=np.stack((itemX,itemY),axis=-1)[:20000]
num_poi=locations.shape[0]

adjacency_list=CreateAdjacencyList(locations)
locations = torch.tensor(location, dtype=torch.float32).to(device)

CnitemC=np.concatenate([itemC, np.arange(num_category)])
CnitemC=torch.tensor(CnitemC).to(device)

edge_index,turnidx= adjacency_list_to_edge_index(adjacency_list,CnitemC,num_poi)

Ec=Ec.to(device)

node_feats = PositionEmbedding(location,)


# 模型与优化器
model = HeteroGraphModel(node_dim=node_dim, cat_dim=cat_dim ,hidden_dim=hidden_dim, edge_dim=edge_dim, output_dim=output_dim,CnitemC=CnitemC).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# -------------------------------
# 4. 训练循环
# -------------------------------
for epoch in range(100):
    optimizer.zero_grad()
    node_embeds, edge_weights = model(node_feats, edge_index.to(device),turnidx, locations,Ec)
    # 你可以在这里添加损失函数，例如基于边权重或嵌入对比损失
    loss = torch.sum(edge_weights)  
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


In [ ]:
print(edge_index)

### Venue Category Analysis

# 负采样优化
空间融合基本方法：基于流行度采样之上，候选负样本缩小到用户活动范围内，目的是排除活动范围外流行的伪负样本
活动范围：用户交互过的地点位置取最大外切矩形
分层采样：依据交互次数排行 找到频繁访问区域作为生活区 最少访问区域作为危险区 其余部分作为探索区
假设1：用户熟知生活区所有地点，因此没有访问的地点作为负样本
假设2：危险区可能是用户真的不感兴趣的区域 也有可能是不方便涉足的区域，可以根据危险区和生活区的距离进行从分段处理控制采样比例 危险区采样比例根据其与生活区距离
探索区域：融合类目分布进行采样


物品的特征可以这样观察 冷门-流行 丰富-稀有 再加上用户访问次数特征 构建出一个三维空间
丰富稀有可以转化成 日常化程度 
#### 假设1：相比更日常的地点人们对稀有的地点更感兴趣
#### 假设2：用户更多访问的地方可能不是因为感兴趣而是因为日常生活需要
#### 假设3：用户的频繁访问点是集中的，用户频繁访问区内的地点用户都熟知，未访问的地点大概率是不感兴趣
#### 假设4：用户频繁访问区外的用户的偶尔访问点是集中的，用户偶尔访问区根据与频繁访问区的距离可以感知用户是对这块区域没兴趣还是不方便探索，这个距离是根据整体区域大小裁定的
#### 假设5：除了偶尔访问区和频繁访问区，剩下的探索区是很未知的，
#### 假设6：根据频繁访问区每个类目 用户访问占区域内总数比例 粗略推断用户对类目的兴趣 把这个权重作用在探索区
#### 策略1：根据全体交互item划分出总区域
#### 策略2：根据分位数划分出频繁访问item，在划分出频繁访问去地理包络，记录包络中心点位置，包络中未访问的项目直接划分成候选负样本
#### 策略3：在频繁访问区地理包络外根据分位数划分出偶尔访问item，得到偶尔访问区地理包络，以中心位置到频繁访问区中心位置距离和总区域斜边两者倒数比来度量相近程度作为偶尔访问区采样权重



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

POI_interaction_matrix,category_interaction_matrix,category_ids_counts=counting4all(dataset,device)

reciprocalrarity=reciprocal_rarity(category_ids_counts)
exprarity=exp_rarity(category_ids_counts,device)

#### 现象1：活跃用户和非活跃用户的交互分布特征相似 大多数访问的地点去的次数很少 只有少数地点去了很多次 这些地点反映用户兴趣

In [ ]:
import torch
import plotly.express as px
import numpy as np

# 假设 interaction_matrix 是给定的张量
# Step 1: Identify the most active user (column with max interactions excluding the first row and column)
active_user_idx = category_interaction_matrix[1:, 1:].sum(1).argmax()
active_user = category_interaction_matrix[active_user_idx+1,:].cpu().numpy()

# Step 2: Identify the least active user (column with min interactions excluding the first row and column)
inactive_user_idx = category_interaction_matrix[1:, 1:].sum(1).argmin()
inactive_user = category_interaction_matrix[inactive_user_idx+1,:].cpu().numpy()

# 计算均值
mean_interaction = category_interaction_matrix[1:,1:].mean(dtype=torch.float32)
ordinary_user_idx = (abs(category_interaction_matrix[1:,1:] - mean_interaction)).argmin()
ordinary_user=category_interaction_matrix[ordinary_user_idx+1,:].cpu().numpy()

# Step 3: Filter out zeros
active_user = active_user[active_user > 0]
inactive_user = inactive_user[inactive_user > 0]
ordinary_user=ordinary_user[ordinary_user>0]

# Step 4: Create DataFrames for Plotly
active_df = pd.DataFrame({"Interactions": active_user, "User Type": "Active User"})
inactive_df = pd.DataFrame({"Interactions": inactive_user, "User Type": "Inactive User"})
ordinary_df= pd.DataFrame({"Interactions": ordinary_user, "User Type": "Ordinary User"})

# Combine both for easier plotting
combined_df = pd.concat([active_df, inactive_df,ordinary_df])
print(active_user.sum(),inactive_user.sum(),ordinary_user.sum())
# Step 5: Plot histograms using Plotly

fig = px.histogram(
    combined_df,
    x="Interactions",
    color="User Type",
    title=(
      "How can users' interests be effectively measured,<br>"
      "while balancing the trade-off between interest breadth and depth,<br>"
      "as illustrated by interaction quantities of <br>"
      "User-Category and Category-POI (user-wise) or User-POI?"),
    labels={"Interactions": "Number of Interactions", "User Type": "User Type"},
    nbins=100,  # Adjust the number of bins as needed
)

# Customize the title position and style
fig.update_layout(
    title={
        "y": 0.7,  # Adjust vertical position (1.0 is top, 0.0 is bottom)
        "x": 0.5,   # Adjust horizontal position (0.5 is centered)
        "xanchor": "center",  # Anchor to center
        "yanchor": "top",     # Anchor to top
        "font": {"size": 16},  # Adjust font size
    }
)


fig.show()


问题：如何区分大量的少交互地点进行兴趣度量 实际上访问次数极多的地点不具有参考价值 它们可能是日常所需，不能很好地反映兴趣
方案：把地点访问次数转化成类目访问次数 
首先把userid-itemid交互序列转化成userid-categoryid交互序列进行归并 这样地分布更加平滑，访问总次数是一样的，只不过把大量只访问一次的地点归并成访问多次的类目。 
这样我们就可以区分访问次数极少的地点 

基于类目的兴趣度量：
假设：偶尔访问类目和频繁访问类目不能反映兴趣
定义用户兴趣类目：排除极端访问次数类目后的类目

问题：用户对访问次数相同的类目兴趣度一致吗？应该怎么区分？
方案：基于类目稀有度的加权，同样的访问次数，用户很可能对更稀有的类目感兴趣，引入类目稀有度矩阵

后续模型修改：
引入兴趣广度概念：不同用户以同样次数访问同个地点，反映的兴趣度是不同的，应该依赖用户访问总类目数量进行加权度量，得到更合理的用户-地点相似度

问题：如何把类目数量权重映射到稀有度上？
直接映射：倒数加权，这样的得到的稀有度是突变的 我希望稀有度

In [ ]:
import plotly.graph_objects as go
import torch


reciprocal_rarity=reciprocalrarity.cpu()
exp_rarity=exprarity.cpu()

# 创建图表对象
fig = go.Figure()

# Reciprocal Rarity Histogram
fig.add_trace(go.Histogram(
    x=reciprocal_rarity[1:],
    nbinsx=1000,
    name="Reciprocal Rarity",
    marker_color="skyblue",
    histnorm='probability',
    opacity=0.7,
    showlegend=True  # 不显示图例
))

# Log Rarity Histogram
fig.add_trace(go.Histogram(
    x=exp_rarity[1:],
    nbinsx=500,
    name="Exp Rarity",
    marker_color="lightgreen",
    opacity=0.7,
    histnorm='probability',
    showlegend=True  # 不显示图例
))

# 计算方差和峰度
reciprocal_var = torch.var(reciprocal_rarity[1:])
reciprocal_kurt = kurtosis(reciprocal_rarity[1:])
exp_var = torch.var(exp_rarity[1:])
exp_kurt = kurtosis(exp_rarity[1:])


# 添加外部文本框显示方差和峰度
fig.add_annotation(
    x=0.5,  # x轴位置，控制文本框水平位置
    y=1.15,  # y轴位置，调整文本框在图表的外部
    text=f"Reciprocal Rarity\nVar: {reciprocal_var:.4e}, Kurtosis: {reciprocal_kurt:.4e}",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center",
    bgcolor="white",
    opacity=0.8,
    xref="paper",  # 使用 'paper' 坐标系，让文本框不受图表范围限制
    yref="paper"   # 使用 'paper' 坐标系，使文本框在图表外
)

fig.add_annotation(
    x=0.5,  # x轴位置，控制文本框水平位置
    y=1.05,  # y轴位置，调整文本框在图表的外部
    text=f"Exp Rarity\nVar: {exp_var:.4e}, Kurtosis: {exp_kurt:.4e}",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center",
    bgcolor="white",
    opacity=0.8,
    xref="paper",  # 使用 'paper' 坐标系，放置在图表外
    yref="paper"   # 使用 'paper' 坐标系，放置在图表外
)

# 更新布局
fig.update_layout(
    title="Histograms of Reciprocal and Log Rarity",
    xaxis_title="Value",
    yaxis_title="Frequency",
    title_font_size=20,
    barmode="overlay",  # 使用 "overlay" 叠加直方图，"group" 可并排显示
    showlegend=True    # 隐藏图例
)

# 显示图表
fig.show()#5.96

In [ ]:
import torch
import numpy as np
import pandas as pd


# 2. 将数据转为 PyTorch Tensor 并移动到 GPU 上
venue_id = torch.tensor(venue_id, dtype=torch.long).cuda()  
venue_category = torch.tensor(venue_category, dtype=torch.long).cuda()
x = torch.tensor(x, dtype=torch.float32).cuda()  
y = torch.tensor(y, dtype=torch.float32).cuda()

# 3. 设置网格的尺寸（例如：每个网格大小为500x500）
grid_width = 500  # 网格宽度
grid_height = 500  # 网格高度

# 4. 计算网格的数量（基于坐标的最大最小值）
x_min, x_max = x.min(), x.max()
y_min, y_max = y.min(), y.max()

# 计算网格行列数
num_x_grids = int((x_max - x_min) // grid_width) + 1
num_y_grids = int((y_max - y_min) // grid_height) + 1


# 5. 分批次处理
batch_size = 10000  # 每批处理的样本数量
num_batches = len(venue_id) // batch_size + 1  # 批次数量

# 创建稠密张量来暂存更新，最后将其转换为稀疏张量
venue_density_matrix = torch.zeros((num_y_grids, num_x_grids), dtype=torch.int32).cuda()
category_density_matrix = torch.zeros((num_y_grids, num_x_grids, len(torch.unique(venue_category))), dtype=torch.int32).cuda()

for batch_idx in range(num_batches):
    # 计算每个批次的索引范围
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(venue_id))
    
    # 获取当前批次数据
    batch_venue_id = venue_id[start_idx:end_idx]
    batch_venue_category = venue_category[start_idx:end_idx]
    batch_x = x[start_idx:end_idx]
    batch_y = y[start_idx:end_idx]

    # 计算每个数据点所属的网格
    grid_x = ((batch_x - x_min) / grid_width).floor().long()  # 计算 x 对应的网格位置
    grid_y = ((batch_y - y_min) / grid_height).floor().long()  # 计算 y 对应的网格位置

    # 确保网格索引不超出边界
    grid_x = torch.clamp(grid_x, 0, num_x_grids - 1)
    grid_y = torch.clamp(grid_y, 0, num_y_grids - 1)

    # 使用 scatter_add_ 更新稠密张量的计数
    venue_density_matrix.index_put_((grid_y, grid_x), torch.ones(len(grid_y), dtype=torch.int32).cuda(), accumulate=True)
    
    # 更新类别密度矩阵
    for i in range(len(batch_venue_category)):
        category_density_matrix[grid_y[i], grid_x[i], batch_venue_category[i]] += 1

# 7. 将稠密张量转换为稀疏张量
venue_density_matrix_sparse = venue_density_matrix.to_sparse()
category_density_matrix_sparse = category_density_matrix.to_sparse()

# 8. 打印结果
print("Venue Density Matrix Sparse:")
print(venue_density_matrix_sparse)
print("Category Density Matrix Sparse:")
print(category_density_matrix_sparse)


In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc
import torch

# 假设 venue_density_matrix_sparse 是一个 PyTorch tensor
# 你可以根据你实际的场景替换这个变量
venue_density_matrix_sparse = torch.rand(num_y_grids, num_x_grids)  # 模拟密度矩阵

# 将 x_min 和 y_min 移回 CPU
xmin = x_min.cpu().item()
ymin = y_min.cpu().item()

# 创建空的图形对象
fig = go.Figure()

# 定义颜色范围，这里选择一个颜色调色板，如蓝色渐变
color_scale = 'Blues'  # 可以选择其他颜色范围，如 'Viridis', 'Cividis', 'Inferno'

# 使用 PyTorch 的 max 和 min 函数
max_density = torch.max(venue_density_matrix_sparse).item()  # 获取最大密度
min_density = torch.min(venue_density_matrix_sparse).item()  # 获取最小密度
normalized_density = (venue_density_matrix_sparse - min_density) / (max_density - min_density)  # 归一化

# 添加网格图层：根据密度值使用颜色
for i in range(num_y_grids):
    for j in range(num_x_grids):
        # 计算网格的边界
        grid_x_min = xmin + j * grid_width
        grid_x_max = grid_x_min + grid_width
        grid_y_min = ymin + i * grid_height
        grid_y_max = grid_y_min + grid_height
        
        # 根据网格的密度值计算颜色
        density = normalized_density[i, j].item()  # 转换为 Python 数字
        # 使用颜色渐变进行映射
        fillcolor = pc.sequential.Blues[int(density * (len(pc.sequential.Blues) - 1))]

        # 绘制矩形网格，设置透明度
        fig.add_trace(go.Scatter(
            x=[grid_x_min, grid_x_min, grid_x_max, grid_x_max, grid_x_min],
            y=[grid_y_min, grid_y_max, grid_y_max, grid_y_min, grid_y_min],
            fill='toself',
            fillcolor=fillcolor,
            line=dict(width=1, color='black'),  # 添加黑色网格线
            mode='lines',
            name='Grid',  # 给网格设置名字，用于筛选
            visible=True  # 默认显示
        ))

# 创建一个调色板来表示 category_id 的颜色
category_colors = pc.qualitative.Set1
marker_colors = [category_colors[c % len(category_colors)] for c in venue_category.cpu().numpy()]

# 添加数据点图层
fig.add_trace(go.Scattergl(
    x=x.cpu().numpy(),
    y=y.cpu().numpy(),
    mode='markers',
    marker=dict(
        size=5,
        color=marker_colors,  # 使用 category_id 作为颜色
        showscale=True  # 显示颜色条
    ),
    name='Data Points',  # 给数据点设置名字，用于筛选
    visible=True  # 默认显示
))

# 设置布局
fig.update_layout(
    title="Venue Points with Density-based Coloring and Grid Lines",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    showlegend=True,
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [None, {'visible': [True, False]}],  # 隐藏数据点，只显示网格
                    'label': 'Show Grid Only',
                    'method': 'relayout'
                },
                {
                    'args': [None, {'visible': [False, True]}],  # 隐藏网格，只显示数据点
                    'label': 'Show Data Points Only',
                    'method': 'relayout'
                },
                {
                    'args': [None, {'visible': [True, True]}],  # 显示所有
                    'label': 'Show Both',
                    'method': 'relayout'
                }
            ],
            'direction': 'down',
            'showactive': True,
            'x': 0.17,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }
    ]
)

# 在浏览器中显示
fig.show(renderer="browser")


In [ ]:
print(test_data._dataset.inter_feat)

地点：偏远中心稀有丰富
街区：偏远中心密集稀疏
用500m*500m作为基本地理块进行聚合
街区密度阈值 街区偏远度阈值
中心和密集相关
丰富地点一定是便利店之类的常用地点
重点是稀有地点区分
偏远但密集的是乡镇 偏远密集街区的丰富地点是便民设施 稀有地点大概率是学校 政府等大场地 小概率是景点
偏远稀疏的是无人区 偏远稀疏街区的丰富地点可能就是便利店 但是稀有地点很可能是景点
中心稀疏是城郊 景点的概率较大
中心密集的是市中心 这是推荐的重点

In [ ]:
for batch in train_data:
    print(batch['user_ids'])
    break

# Mamba4POI

In [ ]:
import sys
import logging
from logging import getLogger
from recbole.utils import init_logger, init_seed
from Modules.mamba4poi import Mamba4POI
from recbole.config import Config
from utils import *
from recbole.trainer import Trainer
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
    set_color,
    get_flops,
    get_environment,
)
import torch
from Modules.myutils import * 

config = Config(model=Mamba4POI, config_file_list=['config.yaml'])
dataset = create_dataset(config)
train_data,valid_data,test_data = data_preparation(config, dataset)


In [ ]:
from Modules.mamba4poi import Mamba4POI
from utils import *
import os
from recbole.trainer import Trainer
if __name__ == '__main__':
    torch.cuda.empty_cache()
    config = Config(model=Mamba4POI, config_file_list=['config.yaml'])
    init_seed(config['seed'], config['reproducibility'])
    
    # logger initialization
    init_logger(config)
    logger = getLogger()
    logger.info(sys.argv)
    logger.info(config)

    logger.info(dataset)

    # model loading and initialization
    init_seed(config["seed"] + config["local_rank"], config["reproducibility"])
    model = Mamba4POI(config, train_data.dataset).to(config['device'])
    logger.info(model)
    
    transform = construct_transform(config)
    flops = get_flops(model, dataset, config["device"], logger, transform)
    logger.info(set_color("FLOPs", "blue") + f": {flops}")

    # trainer loading and initialization
    trainer = Trainer(config, model)

    best_valid_score, best_valid_result = trainer.fit(
    train_data,
    valid_data,  # 可以保留验证数据集
    verbose=True,    # 保留详细信息，打印结果
    saved=True,      # 根据需要决定是否保存模型参数
    show_progress=True,
    callback_fn=None  # 如果不需要回调函数，可以设置为 None
)



    # model evaluation
    test_result = trainer.evaluate(
        test_data, show_progress=config["show_progress"]
    )
    
    environment_tb = get_environment(config)
    logger.info(
        "The running environment of this training is as follows:\n"
        + environment_tb.draw()
    )

    logger.info(set_color("best valid ", "yellow") + f": {best_valid_result}")
    logger.info(set_color("test result", "yellow") + f": {test_result}")

# SRGNN

In [1]:
import sys
import logging
from logging import getLogger
from recbole.utils import init_logger, init_seed
from recbole.model.sequential_recommender import SRGNN
from recbole.config import Config
from utils import *
from recbole.trainer import Trainer
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
    set_color,
    get_flops,
    get_environment,
)
import torch
from Modules.myutils import * 

config = Config(model=SRGNN, config_file_list=['SRGNN.yaml'])
dataset = create_dataset(config)
train_data,valid_data,test_data = data_preparation(config, dataset)


In [ ]:
from recbole.model.sequential_recommender import SRGNN
from utils import *
import os
from recbole.trainer import Trainer
if __name__ == '__main__':
    torch.cuda.empty_cache()
    config = Config(model=SRGNN, config_file_list=['SRGNN.yaml'])
    init_seed(config['seed'], config['reproducibility'])
    
    # logger initialization
    init_logger(config)
    logger = getLogger()
    logger.info(sys.argv)
    logger.info(config)

    logger.info(dataset)

    # model loading and initialization
    init_seed(config["seed"] + config["local_rank"], config["reproducibility"])
    model = SRGNN(config, train_data.dataset).to(config['device'])
    logger.info(model)
    
    transform = construct_transform(config)
    flops = get_flops(model, dataset, config["device"], logger, transform)
    logger.info(set_color("FLOPs", "blue") + f": {flops}")

    # trainer loading and initialization
    trainer = Trainer(config, model)

    best_valid_score, best_valid_result = trainer.fit(
    train_data,
    valid_data,  # 可以保留验证数据集
    verbose=True,    # 保留详细信息，打印结果
    saved=True,      # 根据需要决定是否保存模型参数
    show_progress=True,
    callback_fn=None  # 如果不需要回调函数，可以设置为 None
)



    # model evaluation
    test_result = trainer.evaluate(
        test_data, show_progress=config["show_progress"]
    )
    
    environment_tb = get_environment(config)
    logger.info(
        "The running environment of this training is as follows:\n"
        + environment_tb.draw()
    )

    logger.info(set_color("best valid ", "yellow") + f": {best_valid_result}")
    logger.info(set_color("test result", "yellow") + f": {test_result}")

16 Dec 00:44    INFO  ['/home/chillypepper/anaconda3/envs/mamba4rec/lib/python3.8/site-packages/ipykernel_launcher.py', '--f=/home/chillypepper/.local/share/jupyter/runtime/kernel-v31a7f642571cce262998cb182df6de5d4ea1299f0.json']
16 Dec 00:44    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = dataset/foursquare_NYC
checkpoint_dir = SavedData
show_progress = True
save_dataset = True
dataset_save_path = SavedData/foursquare_NYC-FourSquare.pth
save_dataloaders = True
dataloaders_save_path = 
log_wandb = True

Training Hyper Parameters:
epochs = 200
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.01
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'g

Train     0:   0%|                                                           | 0/45 [00:00<?, ?it/s]:   0%|                                   | 0/45 [00:01<?, ?it/s, GPU RAM: 4.12 G/5.79 G]:   2%|▌                          | 1/45 [00:01<01:27,  1.99s/it, GPU RAM: 4.12 G/5.79 G]:   2%|▌                          | 1/45 [00:03<01:27,  1.99s/it, GPU RAM: 4.71 G/5.79 G]:   4%|█▏                         | 2/45 [00:03<01:23,  1.94s/it, GPU RAM: 4.71 G/5.79 G]:   4%|█▏                         | 2/45 [00:05<01:23,  1.94s/it, GPU RAM: 4.71 G/5.79 G]:   7%|█▊                         | 3/45 [00:05<01:20,  1.92s/it, GPU RAM: 4.71 G/5.79 G]:   7%|█▊                         | 3/45 [00:07<01:20,  1.92s/it, GPU RAM: 4.71 G/5.79 G]:   9%|██▍                        | 4/45 [00:07<01:18,  1.92s/it, GPU RAM: 4.71 G/5.79 G]:   9%|██▍                        | 4/45 [00:09<01:18,  1.92s/it, GPU RAM: 4.71 G/5.79 G]:  11%|███                        | 5/45 [00:09<01:16,  1.91s/it, GPU RAM: 4.71 G/5.79 G]:  11%|███

# Bert4rec

In [ ]:
import sys
import logging
from logging import getLogger
from recbole.utils import init_logger, init_seed
from recbole.model.sequential_recommender import BERT4Rec
from recbole.config import Config
from utils import *
from recbole.trainer import Trainer
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
    set_color,
    get_flops,
    get_environment,
)
import torch
from Modules.myutils import * 

config = Config(model=BERT4Rec, config_file_list=['BERT4Rec.yaml'])
dataset = create_dataset(config)
train_data,valid_data,test_data = data_preparation(config, dataset)


In [ ]:
from recbole.model.sequential_recommender import BERT4Rec
from utils import *
import os
from recbole.trainer import Trainer
if __name__ == '__main__':
    torch.cuda.empty_cache()
    config = Config(model=BERT4Rec, config_file_list=['BERT4Rec.yaml'])
    init_seed(config['seed'], config['reproducibility'])
    
    # logger initialization
    init_logger(config)
    logger = getLogger()
    logger.info(sys.argv)
    logger.info(config)

    logger.info(dataset)

    # model loading and initialization
    init_seed(config["seed"] + config["local_rank"], config["reproducibility"])
    model = BERT4Rec(config, train_data.dataset).to(config['device'])
    logger.info(model)
    
    transform = construct_transform(config)
    flops = get_flops(model, dataset, config["device"], logger, transform)
    logger.info(set_color("FLOPs", "blue") + f": {flops}")

    # trainer loading and initialization
    trainer = Trainer(config, model)

    best_valid_score, best_valid_result = trainer.fit(
    train_data,
    valid_data,  # 可以保留验证数据集
    verbose=True,    # 保留详细信息，打印结果
    saved=True,      # 根据需要决定是否保存模型参数
    show_progress=True,
    callback_fn=None  # 如果不需要回调函数，可以设置为 None
)



    # model evaluation
    test_result = trainer.evaluate(
        test_data, show_progress=config["show_progress"]
    )
    
    environment_tb = get_environment(config)
    logger.info(
        "The running environment of this training is as follows:\n"
        + environment_tb.draw()
    )

    logger.info(set_color("best valid ", "yellow") + f": {best_valid_result}")
    logger.info(set_color("test result", "yellow") + f": {test_result}")

In [ ]:
import re
import matplotlib.pyplot as plt

def extract_metrics_from_log(log_file):
    # 初始化指标
    epochs = []
    train_losses = []
    valid_scores = []
    recall_1 = []
    recall_5 = []
    recall_10 = []
    recall_20 = []
    mrr_1 = []
    mrr_5 = []
    mrr_10 = []
    mrr_20 = []
    ndcg_1 = []
    ndcg_5 = []
    ndcg_10 = []
    ndcg_20 = []
    current_epoch = -1
    with open(log_file, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            # 提取训练损失和验证得分
            train_loss_match = re.search(r'train loss: ([\d\.]+)', line)
            if train_loss_match:
                train_losses.append(float(train_loss_match.group(1)))

            valid_score_match = re.search(r'valid_score: ([\d\.]+)', line)
            if valid_score_match:
                valid_scores.append(float(valid_score_match.group(1)))

            # 提取 recall、mrr、ndcg 等指标
            result_match = re.search(r'recall@1 : ([\d\.]+).*?recall@5 : ([\d\.]+).*?recall@10 : ([\d\.]+).*?recall@20 : ([\d\.]+).*?mrr@1 : ([\d\.]+).*?mrr@5 : ([\d\.]+).*?mrr@10 : ([\d\.]+).*?mrr@20 : ([\d\.]+).*?ndcg@1 : ([\d\.]+).*?ndcg@5 : ([\d\.]+).*?ndcg@10 : ([\d\.]+).*?ndcg@20 : ([\d\.]+)', line)
            if result_match:
                recall_1.append(float(result_match.group(1)))
                recall_5.append(float(result_match.group(2)))
                recall_10.append(float(result_match.group(3)))
                recall_20.append(float(result_match.group(4)))
                mrr_1.append(float(result_match.group(5)))
                mrr_5.append(float(result_match.group(6)))
                mrr_10.append(float(result_match.group(7)))
                mrr_20.append(float(result_match.group(8)))
                ndcg_1.append(float(result_match.group(9)))
                ndcg_5.append(float(result_match.group(10)))
                ndcg_10.append(float(result_match.group(11)))
                ndcg_20.append(float(result_match.group(12)))

              # 提取 epoch 信息，避免重复记录
            epoch_match = re.search(r'epoch (\d+)', line)
            if epoch_match:
                epoch = int(epoch_match.group(1))
                if epoch != current_epoch:  # 如果当前epoch和上一条记录的epoch不同，才记录
                    epochs.append(epoch)
                    current_epoch = epoch
                
    return {
        'epochs': epochs,
        'train_losses': train_losses,
        'valid_scores': valid_scores,
        'recall_1': recall_1,
        'recall_5': recall_5,
        'recall_10': recall_10,
        'recall_20': recall_20,
        'mrr_1': mrr_1,
        'mrr_5': mrr_5,
        'mrr_10': mrr_10,
        'mrr_20': mrr_20,
        'ndcg_1': ndcg_1,
        'ndcg_5': ndcg_5,
        'ndcg_10': ndcg_10,
        'ndcg_20': ndcg_20,
    }

def plot_metrics(metrics):
    # 绘制训练损失和验证得分
    plt.figure(figsize=(15, 10))

    # 训练损失
    plt.subplot(2, 2, 1)
    plt.plot(metrics['epochs'], metrics['train_losses'], label='Train Loss', marker='o', markersize=4)
    plt.xlabel('Epoch')
    plt.ylabel('Train Loss')
    plt.title('Train Loss')
    plt.grid(True)

    # 验证得分
    plt.subplot(2, 2, 2)
    plt.plot(metrics['epochs'], metrics['valid_scores'], label='Valid Score', marker='o', markersize=4)
    plt.xlabel('Epoch')
    plt.ylabel('Valid Score')
    plt.title('Valid Score')
    plt.grid(True)

    # 绘制Recall@1, Recall@5, Recall@10, Recall@20
    plt.subplot(2, 2, 3)
    plt.plot(metrics['epochs'], metrics['recall_1'], label='Recall@1', marker='o', markersize=4)
    plt.plot(metrics['epochs'], metrics['recall_5'], label='Recall@5', marker='o', markersize=4)
    plt.plot(metrics['epochs'], metrics['recall_10'], label='Recall@10', marker='o', markersize=4)
    plt.plot(metrics['epochs'], metrics['recall_20'], label='Recall@20', marker='o', markersize=4)
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.title('Recall@1, Recall@5, Recall@10, Recall@20')
    plt.legend()
    plt.grid(True)

    # 绘制MRR@1, MRR@5, MRR@10, MRR@20
    plt.subplot(2, 2, 4)
    plt.plot(metrics['epochs'], metrics['mrr_1'], label='MRR@1', marker='o', markersize=4)
    plt.plot(metrics['epochs'], metrics['mrr_5'], label='MRR@5', marker='o', markersize=4)
    plt.plot(metrics['epochs'], metrics['mrr_10'], label='MRR@10', marker='o', markersize=4)
    plt.plot(metrics['epochs'], metrics['mrr_20'], label='MRR@20', marker='o', markersize=4)
    plt.xlabel('Epoch')
    plt.ylabel('MRR')
    plt.title('MRR@1, MRR@5, MRR@10, MRR@20')
    plt.legend()
    plt.grid(True)

    # 显示图表
    plt.tight_layout()
    plt.show()

log_file_path = '/mnt/nvme0n1p2/Files/Code/Mamba/Mamba4POI/log/Mamba4POI/Valid/Location User Negsampler.log'  # 替换为你的日志文件路径
metrics = extract_metrics_from_log(log_file_path)

plot_metrics(metrics)
